# PRISM Quickstart Tutorial

Welcome to PRISM (Phase Retrieval Imaging using Sparse Measurements)!

This tutorial will walk you through:
1. Setting up a basic simulation
2. Understanding instrument presets
3. Visualizing optical imaging
4. Understanding resolution limits

**Estimated time**: 15-20 minutes

## 1. Setup and Imports

In [ ]:
# Standard imports
import matplotlib.pyplot as plt
import numpy as np
import torch

# PRISM imports
from prism.config.constants import um
from prism.core import create_usaf_target
from prism.core.instruments import create_instrument
from prism.core.patterns import generate_fermat_spiral
from prism.scenarios import get_scenario_preset

# Notebook settings
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["figure.dpi"] = 100

print("Imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Choose a Scenario Preset

PRISM provides several pre-configured scenario presets for common use cases:

In [ ]:
# Available presets include:
# - "microscope_40x_air": 40x/0.9 NA microscope
# - "microscope_100x_oil": 100x/1.4 NA oil immersion
# - "drone_100m": Drone at 100m altitude
# - "drone_500m": Drone at 500m altitude

# Load a microscope preset
scenario = get_scenario_preset("microscope_40x_air")

print("Scenario Configuration:")
print(f"  Name: {scenario.name}")
print(f"  Objective: {scenario.objective_spec}")
print(f"  Illumination Mode: {scenario.illumination_mode}")
print(f"  Wavelength: {scenario.wavelength * 1e9:.0f} nm")
print(f"  Lateral Resolution: {scenario.lateral_resolution_nm:.0f} nm")
print(f"  Field of View: {scenario.field_of_view_um:.1f} um")

## 3. Create the Instrument

Convert the scenario to an actual instrument we can use for imaging:

In [ ]:
# Convert scenario to instrument configuration
instrument_config = scenario.to_instrument_config()

# Create the actual instrument
microscope = create_instrument(instrument_config)

print("Instrument Created:")
print(f"  Type: {type(microscope).__name__}")
print(f"  Resolution Limit: {microscope.resolution_limit * 1e9:.0f} nm")
info = microscope.get_info()
print(f"  Object Pixel Size: {info['object_pixel_size'] * 1e9:.1f} nm")

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

## 4. Generate Sampling Pattern

Visualize where measurements will be taken using a Fermat spiral pattern:

In [ ]:
# Configuration
n_samples = 30  # Number of measurement positions
scan_radius = 50  # How far to scan from center (pixels)

# Generate Fermat spiral sampling points
# Note: generate_fermat_spiral returns (n_points, 1 or length, 2) tensor
sample_points = generate_fermat_spiral(n_points=n_samples, r_max=scan_radius, length=0)
sample_points = sample_points.squeeze().numpy()  # Shape: (n_samples, 2)

# Visualize
plt.figure(figsize=(8, 8))
scatter = plt.scatter(
    sample_points[:, 0], sample_points[:, 1], 
    c=range(len(sample_points)), cmap="viridis", s=100
)
plt.colorbar(scatter, label="Sample order")
plt.title(f"Sampling Pattern: Fermat Spiral ({n_samples} points)")
plt.xlabel("X position (pixels)")
plt.ylabel("Y position (pixels)")
plt.axis("equal")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Generated {len(sample_points)} sampling points")
print(f"Scan radius: {scan_radius} pixels")

## 5. Create a Test Target

We'll create a USAF-1951 resolution test target to evaluate the optical system:

In [ ]:
# Create USAF-1951 target matching the microscope's field of view
field_size = scenario.field_of_view_um * um  # Convert to meters
resolution = scenario.n_pixels  # Grid resolution (default 1024)

target = create_usaf_target(
    field_size=field_size,
    resolution=resolution,
    groups=None,  # Auto-select groups based on field size
    margin_ratio=0.25,  # 25% margin for proper diffraction simulation
)

# Generate ground truth pattern
ground_truth = target.generate()

print("Target Properties:")
print(f"  Field size: {field_size * 1e6:.0f} um")
print(f"  Resolution: {resolution} x {resolution} pixels")
print(f"  Pixel size: {target.config.pixel_size * 1e9:.1f} nm")

# Visualize
plt.figure(figsize=(10, 10))
extent = [0, field_size * 1e6, 0, field_size * 1e6]  # um
plt.imshow(ground_truth.cpu().numpy(), cmap="gray", extent=extent, origin="lower")
plt.title("USAF-1951 Test Target (Ground Truth)")
plt.xlabel("Position (um)")
plt.ylabel("Position (um)")
plt.colorbar(label="Transmission")
plt.tight_layout()
plt.show()

## 6. Simulate Microscope Measurement

Pass the target through the optical system to see what the microscope actually captures:

In [ ]:
# Prepare input for the forward model (needs batch and channel dims)
input_field = ground_truth.unsqueeze(0).unsqueeze(0)

# Simulate measurement through the microscope
with torch.no_grad():
    measurement = microscope.forward(
        input_field,
        input_mode="intensity",  # Automatically converts via sqrt()
        input_pixel_size=target.config.pixel_size,
    )

# Remove batch/channel dimensions for visualization
measurement_2d = measurement.squeeze().cpu().numpy()

# Compare ground truth vs measurement
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

axes[0].imshow(ground_truth.cpu().numpy(), cmap="gray", extent=extent, origin="lower")
axes[0].set_title("Ground Truth (Ideal)")
axes[0].set_xlabel("Position (um)")
axes[0].set_ylabel("Position (um)")

axes[1].imshow(measurement_2d, cmap="gray", extent=extent, origin="lower")
axes[1].set_title(f"Measurement (Resolution: {scenario.lateral_resolution_nm:.0f} nm)")
axes[1].set_xlabel("Position (um)")
axes[1].set_ylabel("Position (um)")

plt.tight_layout()
plt.show()

print("Observations:")
print("  - Fine features become blurred due to the optical resolution limit")
print(f"  - Features smaller than ~{scenario.lateral_resolution_nm:.0f} nm cannot be resolved")

## 7. Visualize the Point Spread Function (PSF)

The PSF shows how a point source is imaged by the optical system:

In [ ]:
# Compute PSF
psf = microscope.compute_psf(illumination_mode="brightfield")
psf_centered = torch.fft.fftshift(psf).cpu().numpy()

# Get pixel scale
object_pixel_size = info["object_pixel_size"]

# Visualize PSF
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 2D PSF (zoomed to center)
center = psf_centered.shape[0] // 2
crop = 50  # Show central region
psf_crop = psf_centered[center - crop : center + crop, center - crop : center + crop]
crop_extent = crop * object_pixel_size * 1e9  # nm

im = axes[0].imshow(
    psf_crop, cmap="hot", 
    extent=[-crop_extent, crop_extent, -crop_extent, crop_extent]
)
axes[0].set_title("Point Spread Function (2D)")
axes[0].set_xlabel("Position (nm)")
axes[0].set_ylabel("Position (nm)")
plt.colorbar(im, ax=axes[0])

# PSF cross-section
x_coords = (np.arange(psf_centered.shape[0]) - center) * object_pixel_size * 1e9  # nm
psf_line = psf_centered[center, :]
mask = np.abs(x_coords) < 2000  # Show +/- 2000 nm

axes[1].plot(x_coords[mask], psf_line[mask], "r-", linewidth=2)
axes[1].axvline(scenario.lateral_resolution_nm / 2, color="k", linestyle="--", 
                label=f"Resolution: {scenario.lateral_resolution_nm:.0f} nm")
axes[1].axvline(-scenario.lateral_resolution_nm / 2, color="k", linestyle="--")
axes[1].set_title("PSF Cross-Section")
axes[1].set_xlabel("Position (nm)")
axes[1].set_ylabel("Intensity")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"PSF width determines the resolution limit: ~{scenario.lateral_resolution_nm:.0f} nm")

## 8. Next Steps

Try modifying the parameters above:
- **Preset**: Try `"microscope_100x_oil"` for higher resolution
- **Samples**: Increase `n_samples` for more sampling positions
- **Scan radius**: Adjust `scan_radius` for different coverage

For full reconstruction experiments, see:
- [Quickstart 01: Microscopy Basic](quickstart_01_microscopy_basic.ipynb)
- [Quickstart 02: Drone Basic](quickstart_02_drone_basic.ipynb)
- [SPIDS Microscopy Resolution Enhancement](spids_microscopy_resolution_enhancement.ipynb)

For command-line usage:
```bash
uv run python main.py --help
```

## Summary

You've learned:
- How to load scenario presets
- How to create instruments from scenarios
- How to generate sampling patterns
- How to create test targets
- How to simulate optical measurements
- How to visualize the PSF and resolution limit

**Key concepts**:
- Resolution limit is determined by NA and wavelength
- The PSF width shows the smallest resolvable feature
- PRISM can simulate realistic optical measurements

**Next tutorials**:
- Tutorial 2: Pattern design and comparison
- Tutorial 3: Result analysis and metrics